<a href="https://colab.research.google.com/github/Gokul-Mechaktroniker/Interdisciplinary_Project/blob/main/Clara_Schumann_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade langchain openai -q
!pip install unstructured -q
!pip install unstructured[local-inference] -q
!pip install detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2 -q
!apt-get install poppler-utils

In [ ]:
!pip install openai langchain sentence_transformers -q
!pip install chromadb -q

In [ ]:
from langchain.document_loaders import DirectoryLoader

directory = '/content/data'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=1000,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

3831


In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings)


In [ ]:
query = "Wann plant der Absender, am Dienstag aufzustehen, und welche Einladung richtet er aus?"
matching_docs = db.similarity_search(query)

In [ ]:
print(matching_docs[0].page_content)

mir. Er soll Serres nicht ganz vergessen und mich auch einmal mit ein paar Zeilen überraschen (wenn du es erlaubst).


In [ ]:
matching_docs = db.similarity_search_with_score(query,k=5)
matching_docs

[(Document(page_content='mir. Er soll Serres nicht ganz vergessen und mich auch einmal mit ein paar Zeilen überraschen (wenn du es erlaubst).', metadata={'source': '/content/data/cs.txt'}),
  1.0381678342819214),
 (Document(page_content='abstumpfen. Wenn sie mir nicht 24 Franken für 2 Stunden (hintereinander) bezahlen würden, würde ich nicht hingehen.', metadata={'source': '/content/data/cs.txt'}),
  1.0720456838607788),
 (Document(page_content='liebt mich und dich außerordentlich. Nun also keine anderen Gedanken mehr.', metadata={'source': '/content/data/cs.txt'}),
  1.1266183853149414),
 (Document(page_content='Von ganzem Herzen und mit aller unaussprechlichen Liebe küsst dich deine treue Braut Clärchen.', metadata={'source': '/content/data/cs.txt'}),
  1.1647857427597046),
 (Document(page_content='und zu wenig gemütlich. Nicht wahr, du bist einverstanden?', metadata={'source': '/content/data/cs.txt'}),
  1.19193434715271)]

In [ ]:
persist_directory = "chroma_db"

vectordb = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory=persist_directory)

In [ ]:
vectordb.persist()

In [ ]:
new_db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [ ]:
matching_docs = new_db.similarity_search_with_score(query)
matching_docs[0]

(Document(page_content='mir. Er soll Serres nicht ganz vergessen und mich auch einmal mit ein paar Zeilen überraschen (wenn du es erlaubst).', metadata={'source': '/content/data/cs.txt'}),
 1.0381678342819214)

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-srsxrctg" # Enter your openai api key here

In [ ]:
from langchain.chat_models import ChatOpenAI
model_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=model_name)

In [ ]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
query = "In welchem Zeitraum schickte Clara Schumann Briefe an Robert Schumann?"
matching_docs = db.similarity_search(query)
answer = chain.run(input_documents=matching_docs, question=query,)
answer

'Clara Wieck schickte Briefe an Robert Schumann in verschiedenen Zeiträumen. Ein Brief wurde am Montag, den 17. Dezember 1832, von Zwickau nach Leipzig geschickt. Weitere Briefe wurden am Samstag, den 22. und Sonntag, den 23. September 1838, von Zwickau nach Leipzig geschickt. Ein weiterer Brief wurde am Sonntag, den 20. und Montag, den 21. Januar 1839, von Wien nach Stuttgart geschickt. Es ist jedoch möglich, dass es noch weitere Briefe zwischen ihnen gab, von denen wir keine Informationen haben.'

In [ ]:
from langchain.chains import RetrievalQA
retrieval_chain = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=db.as_retriever())
retrieval_chain.run(query)

'Clara Schumann schickte Briefe an Robert Schumann im Zeitraum von Montag, 17. Dezember 1832, Samstag, 22. und Sonntag, 23. September 1838, sowie Sonntag, 20. und Montag, 21. Januar 1839.'